In [10]:
import os, sys
import glob
import pandas as pd
from os import listdir
from os.path import isfile, join
from argparse import ArgumentParser
# Input

#Covarage matrix as csv file without qoutes, seperated by commas
file = "/home/pato/Skrivebord/HPV_subtyping/Andet/covtest.csv"

# Housegenes to check and remove
housegenes = ['PABPN1','SRSF3','PPIE','RAB1B','BTF3']

# Location of all valid reference names that can be given
path = "/home/pato/Skrivebord/HPV_subtyping/References"

# Location of where to save output files
refSavePath = "/home/pato/Skrivebord/HPV_subtyping/scripts/autoDetTest"

In [16]:
# Remove all current inputRefs to avoid appending to them
if len(refSavePath) > 0:
    for f in glob.glob(os.path.join(refSavePath,'*.txt')):
        os.remove(f)

faF = [f for f in listdir(path) if isfile(join(path, f))]
faFiles = []
for i in faF:
    if "fasta.fai" not in i:
        faFiles.append(i)
df = pd.read_csv(file,sep=",")

# Finder samples

In [17]:
excludeCols = ['Sort column', 'Gene', 'contig_srt', 'contig_end', 'Target']
sampleList = []
discardedList = []

for col in df.columns:
  
    if col not in excludeCols:

        # Checking housegenes, first get total cov of them
        hgTot = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            hgTot+= int(hg[col])

        # Now finding outliers in housegene coverage
        notEnoughCov = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            geneCov = int(hg[col])

            # set min cov on housegene to not count as failed
            if geneCov < hgTot*0.1:
                notEnoughCov+=1
            
        # discard sample if too many low housegenes
        if notEnoughCov < 4:
            sampleList.append(col)
        else:
            discardedList.append(col)
discardedList

[]

In [29]:
# Now remove housegenes from samples
ddf = df[~df['Gene'].isin(housegenes)]

In [30]:
ddf

,Sort column,Gene,contig_srt,contig_end,Target,IonXpress_041,IonXpress_042,IonXpress_043,IonXpress_044,IonXpress_045,...,IonXpress_055,IonXpress_056,IonXpress_057,IonXpress_058,IonXpress_059,IonXpress_060,IonXpress_061,IonXpress_062,IonXpress_063,IonXpress_064
5,6,HPV16HR_LCR..E6_1,42,128,AMPL4495518,9134,8287,7471,7766,18115,...,12604,2837,12903,5617,11212,7246,170,2310,13906,418
6,7,HPV16HR_LCR..E6_2,97,183,AMPL4495707,10487,6967,10811,7632,10375,...,11000,6818,11513,6455,9427,8377,159,1891,11168,550
7,8,HPV16HR_E6_1,173,254,AMPL4495482,5058,7160,4220,4814,10440,...,8095,4705,10263,11364,8940,12353,83,1217,8931,334
8,9,HPV16HR_E6_2,262,346,AMPL4495576,1934,1472,1425,889,1737,...,1900,1194,1742,2745,1325,3596,19,103,2253,63
9,10,HPV16HR_E6_3,310,385,AMPL4495636,4001,2922,1304,777,5737,...,7273,2146,11684,11676,8478,8619,55,806,9459,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,174,HPV85pHR_AF131950_E6,5273,5344,AMPL4495581,4403,4101,2614,1473,3970,...,5098,4496,9726,12802,5994,8313,57,495,5633,320
174,175,HPV85pHR_AF131950,5326,5406,AMPL4495498,3518,4281,2118,2256,6991,...,8030,2282,12059,9111,8072,6534,113,1092,8778,274
175,176,HPV85pHR_AF131950,5400,5482,AMPL4495492,4138,5427,2515,2618,7849,...,9909,5870,10398,9560,8997,8418,88,952,12180,386
176,177,HPV85pHR_AF131950,5454,5545,AMPL4495532,3759,4721,2259,1007,8853,...,8465,7443,10596,8338,8335,7662,156,1298,9398,218


In [ ]:
# Look for deletions (possible integrations of the genome) by finding regions of very low coverage
ddf[['Type', 'Gene']] = ddf['Gene'].str.split('_', 1, expand=True)

In [46]:
filtered[sample]

0      385
1      428
2      255
3      135
4      526
      ... 
99     253
100    378
101    317
102    331
103    269
Name: IonXpress_049, Length: 104, dtype: int64

In [ ]:
hpvTypes = pd.unique(ddf['Type'])

for type in hpvTypes:
    filtered = ddf[ddf['Type'] == type]
    print(type)


In [125]:
delJson = {}

for sample in sampleList:
    for hpvtype in hpvTypes:
        filtered = ddf[ddf['Type'] == hpvtype]
        leng = len(filtered[sample])
        sum = filtered[sample].sum()
        
        avg = sum / leng
        if avg > 50:
            cutoff = 50
        else:
            cutoff = avg*0.1

        #for sample in sampleList:#
        filtered = filtered.reset_index()
        lastIdx = 0
        for index, row in filtered.iterrows():
            
            if row[sample] < cutoff:
                # Check if last index contained low cov (potential deletion)
                if (lastIdx == index - 1) or (lastIdx == 0):

                    if 'startpos' not in locals():
                        startpos = row['contig_srt']
                    stoppos = row['contig_end']
                    lastIdx = index

                else:
                    # This is start of a new deletion
                    startpos = row['contig_srt']
                    lastrow = row
                    lastIdx = index


            else:
                # save found startpos and endpos
                if 'startpos' in locals():
                    if 'stoppos' not in locals():
                        stoppos = lastrow['contig_end']

                    # create key if it does not exist:
                    if sample in delJson.keys():
                        pass
                    else:    
                        delJson[sample] = {
                        'del' : []
                        }

                    delJson[sample]['del'].append([hpvtype,startpos,stoppos])

                    #save
                    del startpos, stoppos



In [126]:
delJson

{'IonXpress_041': {'del': [['HPV16HR', 6246, 6325]]},
 'IonXpress_042': {'del': [['HPV16HR', 5532, 5599],
   ['HPV16HR', 6246, 6325],
   ['HPV70', 1056, 1674]]},
 'IonXpress_043': {'del': [['HPV16HR', 5532, 5599],
   ['HPV16HR', 1504, 2647],
   ['HPV16HR', 6246, 6325]]},
 'IonXpress_044': {'del': [['HPV16HR', 5132, 5214],
   ['HPV16HR', 6246, 6325],
   ['HPV85pHR', 5132, 5214]]},
 'IonXpress_046': {'del': [['HPV16HR', 2146, 2208],
   ['HPV16HR', 5532, 5599],
   ['HPV16HR', 6246, 6325],
   ['HPV16HR', 6887, 7039],
   ['HPV70', 7085, 7162],
   ['HPV70', 2146, 2208]]},
 'IonXpress_047': {'del': [['HPV16HR', 5532, 5599], ['HPV16HR', 6246, 6325]]},
 'IonXpress_049': {'del': [['HPV16HR', 1056, 1134],
   ['HPV16HR', 2382, 5599],
   ['HPV16HR', 6246, 6325],
   ['HPV70', 1056, 1134],
   ['HPV67pHR', 2382, 2857],
   ['HPV67pHR', 2903, 3035],
   ['HPV67pHR', 3081, 3422],
   ['HPV11LR', 3450, 3693],
   ['HPV11LR', 3785, 3873],
   ['HPV11LR', 3936, 4458],
   ['HPV85pHR', 4492, 4953],
   ['HPV85pHR'

In [33]:
filtered

,Sort column,Gene,contig_srt,contig_end,Target,IonXpress_041,IonXpress_042,IonXpress_043,IonXpress_044,IonXpress_045,...,IonXpress_056,IonXpress_057,IonXpress_058,IonXpress_059,IonXpress_060,IonXpress_061,IonXpress_062,IonXpress_063,IonXpress_064,Type
163,164,AF131950,4558,4642,AMPL4495503,2659,6048,1205,1506,10432,...,1859,7878,7201,7787,5249,52,1356,9852,76,HPV85pHR
164,165,AF131950,4632,4727,AMPL4495608,1905,2508,1312,1618,3402,...,1620,6057,4001,4309,3063,58,527,5683,25,HPV85pHR
165,166,AF131950,4718,4778,AMPL4495488,19335,20497,11579,13341,25825,...,8861,19626,20063,17653,22970,213,2019,18642,631,HPV85pHR
166,167,AF131950,4779,4869,AMPL4495715,3248,5768,3523,2907,9177,...,5281,8324,8245,8068,8175,236,1379,11579,112,HPV85pHR
167,168,AF131950,4869,4953,AMPL4495587,11575,12921,17558,8352,14532,...,15500,15061,6558,9814,9917,145,366,7132,395,HPV85pHR
168,169,AF131950,4916,4999,AMPL4495489,3738,6710,3981,2229,10811,...,3485,10694,9427,8923,9897,67,1306,11010,160,HPV85pHR
169,170,AF131950_E6,5003,5084,AMPL4495705,2124,1822,1847,1804,1918,...,1545,4533,3992,2470,3146,21,256,2028,19,HPV85pHR
170,171,AF131950_E6,5083,5151,AMPL4495534,6735,10392,2179,2538,20917,...,2701,17180,13222,13799,12657,162,2021,17973,351,HPV85pHR
171,172,AF131950_E6,5132,5214,AMPL4495595,2839,3135,2488,33,3630,...,5067,6450,10146,5346,6276,19,818,5584,3,HPV85pHR
172,173,AF131950_E6,5201,5284,AMPL4495610,2108,1887,1614,1993,2272,...,1462,4627,5086,2627,3827,14,327,3501,111,HPV85pHR


In [7]:
# Now remove all that are not E6 or E7
genesForCovCalc = "_E6|_E7|E6..E7" # Seperated by | 
dfilt = ddf[ddf['Gene'].str.contains(genesForCovCalc)==True]
dfilt

,Sort column,Gene,contig_srt,contig_end,Target,IonXpress_041,IonXpress_042,IonXpress_043,IonXpress_044,IonXpress_045,...,IonXpress_055,IonXpress_056,IonXpress_057,IonXpress_058,IonXpress_059,IonXpress_060,IonXpress_061,IonXpress_062,IonXpress_063,IonXpress_064
7,8,HPV16HR_E6_1,173,254,AMPL4495482,5058,7160,4220,4814,10440,...,8095,4705,10263,11364,8940,12353,83,1217,8931,334
8,9,HPV16HR_E6_2,262,346,AMPL4495576,1934,1472,1425,889,1737,...,1900,1194,1742,2745,1325,3596,19,103,2253,63
9,10,HPV16HR_E6_3,310,385,AMPL4495636,4001,2922,1304,777,5737,...,7273,2146,11684,11676,8478,8619,55,806,9459,165
10,11,HPV16HR_E6_4,392,484,AMPL4495464,4505,3437,1429,186,12467,...,11906,5352,8442,8460,9856,8189,30,1293,16355,140
11,12,HPV16HR_E6_5,433,516,AMPL4495568,9526,10586,9395,7472,18597,...,14303,9959,15371,9083,12695,11634,135,1815,15395,758
12,13,HPV16HR_E6..E7,512,599,AMPL4495558,12580,10488,7972,1902,21549,...,24955,34775,20489,31930,22454,25730,88,2305,35545,749
13,14,HPV16HR_E7_1,560,644,AMPL4495593,4086,6966,2092,2026,14731,...,12652,3870,14736,10062,11036,9508,104,1768,16013,491
14,15,HPV16HR_E7_2,644,733,AMPL4495721,6209,5459,8929,9901,7385,...,6889,7046,13483,4478,6426,5864,124,1141,7838,453
15,16,HPV16HR_E7_3,686,768,AMPL4495494,17342,17640,23511,21689,25313,...,18684,11581,20415,11369,17724,21734,524,2335,23639,3101
16,17,HPV16HR_E7_4,766,851,AMPL4495466,8580,9353,7714,8026,13253,...,9159,5128,12553,4887,9071,6724,245,1399,10716,1467


In [9]:
dfilt[['Type', 'Gene']] = dfilt['Gene'].str.split('_', 1, expand=True)
dfilt

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Sort column,Gene,contig_srt,contig_end,Target,IonXpress_041,IonXpress_042,IonXpress_043,IonXpress_044,IonXpress_045,...,IonXpress_056,IonXpress_057,IonXpress_058,IonXpress_059,IonXpress_060,IonXpress_061,IonXpress_062,IonXpress_063,IonXpress_064,Type
7,8,E6_1,173,254,AMPL4495482,5058,7160,4220,4814,10440,...,4705,10263,11364,8940,12353,83,1217,8931,334,HPV16HR
8,9,E6_2,262,346,AMPL4495576,1934,1472,1425,889,1737,...,1194,1742,2745,1325,3596,19,103,2253,63,HPV16HR
9,10,E6_3,310,385,AMPL4495636,4001,2922,1304,777,5737,...,2146,11684,11676,8478,8619,55,806,9459,165,HPV16HR
10,11,E6_4,392,484,AMPL4495464,4505,3437,1429,186,12467,...,5352,8442,8460,9856,8189,30,1293,16355,140,HPV16HR
11,12,E6_5,433,516,AMPL4495568,9526,10586,9395,7472,18597,...,9959,15371,9083,12695,11634,135,1815,15395,758,HPV16HR
12,13,E6..E7,512,599,AMPL4495558,12580,10488,7972,1902,21549,...,34775,20489,31930,22454,25730,88,2305,35545,749,HPV16HR
13,14,E7_1,560,644,AMPL4495593,4086,6966,2092,2026,14731,...,3870,14736,10062,11036,9508,104,1768,16013,491,HPV16HR
14,15,E7_2,644,733,AMPL4495721,6209,5459,8929,9901,7385,...,7046,13483,4478,6426,5864,124,1141,7838,453,HPV16HR
15,16,E7_3,686,768,AMPL4495494,17342,17640,23511,21689,25313,...,11581,20415,11369,17724,21734,524,2335,23639,3101,HPV16HR
16,17,E7_4,766,851,AMPL4495466,8580,9353,7714,8026,13253,...,5128,12553,4887,9071,6724,245,1399,10716,1467,HPV16HR


In [40]:
#dfilt.IonXpress_041.sum()

In [41]:
# Split i samples og find hvilke HPV typer der passer
hpvTypes = pd.unique(dfilt['Type'])
# For sample, find HPV types in them
covJson = {}
for sample in sampleList:
    covJson.update({sample : {
    }})

    for hpv in hpvTypes:
        filtered = dfilt[dfilt['Type'] == hpv]
        
        # Tilsætter sample, hpvtype og sum af dækning for regioner i hpvtype
        covJson[sample].update({
            hpv : filtered[sample].sum()
            })

In [42]:
covJson

{'IonXpress_041': {'HPV16HR': 105123,
  'HPV70': 17000,
  'HPV67pHR': 38977,
  'HPV11LR': 45600,
  'HPV85pHR': 18209},
 'IonXpress_042': {'HPV16HR': 103960,
  'HPV70': 24829,
  'HPV67pHR': 35590,
  'HPV11LR': 56587,
  'HPV85pHR': 21337},
 'IonXpress_043': {'HPV16HR': 105044,
  'HPV70': 9331,
  'HPV67pHR': 27340,
  'HPV11LR': 34110,
  'HPV85pHR': 10742},
 'IonXpress_044': {'HPV16HR': 104574,
  'HPV70': 8044,
  'HPV67pHR': 23193,
  'HPV11LR': 42077,
  'HPV85pHR': 7841},
 'IonXpress_045': {'HPV16HR': 178815,
  'HPV70': 34777,
  'HPV67pHR': 63795,
  'HPV11LR': 95197,
  'HPV85pHR': 32707},
 'IonXpress_046': {'HPV16HR': 186889,
  'HPV70': 5369,
  'HPV67pHR': 39739,
  'HPV11LR': 16232,
  'HPV85pHR': 2746},
 'IonXpress_047': {'HPV16HR': 105935,
  'HPV70': 5690,
  'HPV67pHR': 20162,
  'HPV11LR': 24615,
  'HPV85pHR': 5754},
 'IonXpress_048': {'HPV16HR': 175115,
  'HPV70': 47747,
  'HPV67pHR': 67397,
  'HPV11LR': 48369,
  'HPV85pHR': 42754},
 'IonXpress_049': {'HPV16HR': 5044,
  'HPV70': 1754,
  

In [43]:
# Tæl sum af dækning over alle regioner for sample, derefter angiv fraktioner

ratioJson = {}


for sample in covJson.keys():
    # Tæl total cov over gener for sample
    sampleCovTotal = 0
    for hpvtype, cov in covJson[sample].items():
        sampleCovTotal+=cov
    # Har nu total, og kan derfor loope igennem igen og lave ratios
    # Sætter sample name:
    ratioJson.update({sample : {
        }})
    
    for hpvtype, cov in covJson[sample].items():
        

        ratioJson[sample].update({
            hpvtype : cov/sampleCovTotal*100
            })


In [44]:
ratioJson

{'IonXpress_041': {'HPV16HR': 46.74023716258576,
  'HPV70': 7.558612594427079,
  'HPV67pHR': 17.330120181940252,
  'HPV11LR': 20.27486672387499,
  'HPV85pHR': 8.096163337171923},
 'IonXpress_042': {'HPV16HR': 42.90495784204075,
  'HPV70': 10.24708732454819,
  'HPV67pHR': 14.688220946500868,
  'HPV11LR': 23.35381732789111,
  'HPV85pHR': 8.80591655901908},
 'IonXpress_043': {'HPV16HR': 56.30363354719752,
  'HPV70': 5.001420401249953,
  'HPV67pHR': 14.654252895742548,
  'HPV11LR': 18.28297608901896,
  'HPV85pHR': 5.757717066791019},
 'IonXpress_044': {'HPV16HR': 56.30461586505069,
  'HPV70': 4.331041463637881,
  'HPV67pHR': 12.48754906342036,
  'HPV11LR': 22.655051176714462,
  'HPV85pHR': 4.221742431176607},
 'IonXpress_045': {'HPV16HR': 44.12015070652938,
  'HPV70': 8.580748153795668,
  'HPV67pHR': 15.740541980947024,
  'HPV11LR': 23.488555136926305,
  'HPV85pHR': 8.07000402180162},
 'IonXpress_046': {'HPV16HR': 74.46518577547565,
  'HPV70': 2.13925689809742,
  'HPV67pHR': 15.83384799282

In [45]:
confirmedJson = {}
for sample in ratioJson.keys():
    
    confirmedJson.update({sample : {
        }})
    
    foundOneForSample = "no"
    highestRatio = 0
    for hpvtype, ratioCov in ratioJson[sample].items():
        
        # Define minimum ratio to confirm hpvtype in sample (1 to 100)
        if ratioCov > 20: 
            confirm = "yes" 
            foundOneForSample = "yes"
            if ratioCov > highestRatio:
                highestRatio = ratioCov
                highestRatioName = hpvtype
        else: 
            confirm = "no"

        confirmedJson[sample].update({
            hpvtype : confirm
            })

        # If no one meets the minimum ratioCov value, set the highest cov as best match
        if foundOneForSample == "no":
            for hpvtype, ratioCov in ratioJson[sample].items():
                if ratioCov > highestRatio:
                    highestRatio = ratioCov
                    highestRatioName = hpvtype
        
            for hpvtype, ratioCov in ratioJson[sample].items():
                if hpvtype == highestRatioName:
                    confirmedJson[sample].update({
                        hpvtype : "yes"
                        })

In [46]:
confirmedJson

{'IonXpress_041': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'yes',
  'HPV85pHR': 'no'},
 'IonXpress_042': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'yes',
  'HPV85pHR': 'no'},
 'IonXpress_043': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'no',
  'HPV85pHR': 'no'},
 'IonXpress_044': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'yes',
  'HPV85pHR': 'no'},
 'IonXpress_045': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'yes',
  'HPV85pHR': 'no'},
 'IonXpress_046': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'no',
  'HPV85pHR': 'no'},
 'IonXpress_047': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'no',
  'HPV85pHR': 'no'},
 'IonXpress_048': {'HPV16HR': 'yes',
  'HPV70': 'no',
  'HPV67pHR': 'no',
  'HPV11LR': 'no',
  'HPV85pHR': 'no'},
 'IonXpress_049': {'HPV16HR': 'yes',
  'HPV70': 'yes',
  'HPV67pHR': 'no',
  'HPV11L

In [ ]:
# For each sample output confirmed hpv types to file
multiFa = []
for sample in confirmedJson.keys():
    for hpvtype, val in confirmedJson[sample].items():
        if val == "yes":

            # convert name to fit fasta files:
            ht = hpvtype.split('_')[1]

            # search list of fasta files for id
            ref = [i for i in faFiles if ht in i]
            # Checking that there is only one match, then saving without .fasta in name
            if len(ref) > 1:
                multiFa.append([sample, ref])
            else:
                ref = ref[0].replace(".fasta","")
                # Append to file
                filename = refSavePath+"/"+sample+".txt"
                f = open(filename, "a")
                f.write(ref+"\n")
                f.close()

# Printing fastas found with multiple hits. Fix these by removing one or more of the files
if len(multiFa) > 0:
    # Append to file
    filename = refSavePath+"/"+sample+"_foundMultiRefsFor.txt"
    f = open(filename, "a")
    f.write(ref+"\n")
    f.close()
